In [1]:
import sys
sys.path.append("..")

In [2]:
from rasterio.warp import calculate_default_transform,reproject,Resampling
from shapely import BufferCapStyle, BufferJoinStyle, buffer
from shapely.geometry import shape, Point, Polygon
from shapely.ops import transform
from osgeo import gdal
from pathlib import Path
import src.lidar_products as lp
import matplotlib.pyplot as plt
import rasterio as rio
import ipywidgets as widgets
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import pdal
import json
import sys
import os


In [3]:
#setting meadow AOI name

name = "Lacey"
#name = "Humbug"
#name = "subb"

In [4]:
# setting output directory and changing working directory to ../data
OUTPUT_DIR = Path(f"../data/{name}")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
#os.chdir("../data")
os.chdir(OUTPUT_DIR)

In [5]:
#shapefile must currently exist in meadow folder
shapefile_path = "meadow_extent.geojson"

In [6]:
intersecting_polys, AOI_EPSG3857_wkt = lp.usgs_3dep_datasets(shapefile_path)

Vector file loaded.
resources.geojson exists in the data folder.
3DEP polygons loaded and projected to Web Mercator (EPSG:3857)
AOI buffered by 10 meters
[('USGS_LPC_CA_NoCAL_Wildfires_B1_2018', <MULTIPOLYGON (((-120.174 38.833, -120.116 38.863, -120.116 38.972, -120.134...>, <MULTIPOLYGON (((-13377749 4697773.7, -13371249 4702103.8, -13371249 4717692...>, 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_CA_NoCAL_Wildfires_B1_2018/ept.json', np.int64(86376910091))]


In [7]:
#wlayer_3DEP_list = []
usgs_3dep_datasets = []
#number_pts_est = []
for i, poly in enumerate(intersecting_polys):
    #wlayer_3DEP = ipyleaflet.WKTLayer(
        #wkt_string=poly[1].wkt, 
        #style={"color": "green"})
    
    #m.add_layer(wlayer_3DEP)
    #wlayer_3DEP_list.append(wlayer_3DEP)
    usgs_3dep_datasets.append(poly[0])
    
#AOI_EPSG3857_wkt = AOI_EPSG3857.wkt

In [8]:
p_pipeline = lp.pdal_pipeline(AOI_EPSG3857_wkt, usgs_3dep_datasets)

In [9]:
pl = pdal.Pipeline(json.dumps(p_pipeline))

In [10]:
%%time
pl.execute()

CPU times: user 4min 24s, sys: 7.42 s, total: 4min 31s
Wall time: 4min 10s


7088795

Lacey - 2m47s, 7088795pts  
Humbug - 15m43s, 25865590pts

## Filling only nodata within meadow boundary

In [11]:
lp.fill_nodata('dtm_temp.tif', shapefile_path)

'Filled dtm_temp.tif and saved to dtm_temp_filled.tif'

In [12]:
lp.fill_nodata('dsm_temp.tif', shapefile_path)

'Filled dsm_temp.tif and saved to dsm_temp_filled.tif'

In [13]:
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(6339)

In [14]:
lp.clip_and_rename_raster('dtm_temp_filled.tif', gdf)

'Saved as: dtm_clipped.tif'

In [15]:
lp.clip_and_rename_raster('dsm_temp_filled.tif', gdf)

'Saved as: dsm_clipped.tif'

In [16]:
lp.chm('dsm_clipped.tif', 'dtm_clipped.tif', gdf)

'Saved as: chm_clipped.tif'

delete all files except _clipped and rename them with location added? ie Lacey_CHM.tif

In [17]:

import glob

# Set your working directory
#directory = "/path/to/your/tif/files"

# Step 1: List all .tif files
tif_files = glob.glob(os.path.join(os.getcwd(), "*.tif"))


In [18]:

for file_path in tif_files:
    filename = os.path.basename(file_path)

    # Step 2: Delete files that do NOT contain "_clipped.tif"
    if "_clipped.tif" not in filename:
        os.remove(file_path)
        print(f"Deleted: {filename}")
        continue

    # Step 3: Rename files ending in "-clipped.tif" to ".tif"
    if filename.endswith("_clipped.tif"):
        new_filename = filename.replace("_clipped.tif", "_" + name + ".tif")
        new_path = os.path.join(os.getcwd(), new_filename)
        os.rename(file_path, new_path)
        print(f"Renamed: {filename} → {new_filename}")


Deleted: dsm_temp.tif
Renamed: dtm_clipped.tif → dtm_Lacey.tif
Renamed: dsm_clipped.tif → dsm_Lacey.tif
Deleted: dtm_temp_filled.tif
Deleted: dsm_temp_filled.tif
Renamed: chm_clipped.tif → chm_Lacey.tif
Deleted: dtm_temp.tif


In [19]:
#move to parent directory
os.chdir("..")
os.getcwd()

'/media/grendel/7db216a7-836f-4e8d-b439-e4f999cedb23/USGS/meadow_assessment/data'